## Проект по курсу Рекоммендательные системы

**Основное**
- Дедлайн - 21 июня 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

### Загрузка необходимых библиотек

In [1]:
# Здаем параметры юпитер ноутбука на авто перзагрузку изменений библиотек в src
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

from catboost import CatBoostClassifier

# Модель второго уровня
#from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k, ap_k, ndcg_at_k, reciprocal_rank_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

C:\Users\vladuz\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### Загрузка тестовых данных

In [3]:
data = pd.read_csv('retail_train.csv')
data_test = pd.read_csv('retail_test.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

### Разбиваем датасет на обучающий и валидацилнный и ранжирующий

In [4]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

VAL_1_WEEKS = 6
VAL_2_WEEKS = 3

In [5]:
# берем данные для тренировки фильтрующей модели 1
data_train_1 = data[data['week_no'] < data['week_no'].max() - (VAL_1_WEEKS + VAL_2_WEEKS)]

# берем данные для валидации фильтрующей модели 1
data_val_1 = data[(data['week_no'] >= data['week_no'].max() - (VAL_1_WEEKS + VAL_2_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_2_WEEKS))]


# берем данные для тренировки ранжирующей модели
data_train_2 = data_val_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ранжирующей модели
data_val_2 = data[data['week_no'] >= data['week_no'].max() - VAL_2_WEEKS]

### Эвристическая фильтрация

In [6]:
n_items_before = data_train_1['item_id'].nunique()

data_train_1 = prefilter_items(data_train_1, item_features, take_n_popular=6000)

n_items_after = data_train_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 6001


In [7]:
# ищем общих пользователей
common_users = data_train_1.user_id.values

data_val_1 = data_val_1[data_val_1.user_id.isin(common_users)]
data_train_2 = data_train_2[data_train_2.user_id.isin(common_users)]
data_val_2 = data_val_2[data_val_2.user_id.isin(common_users)]

In [8]:
result_val_1 = data_val_1.groupby('user_id')['item_id'].unique().reset_index()
result_val_1.columns=['user_id', 'actual_val_1']
result_val_2 = data_val_2.groupby('user_id')['item_id'].unique().reset_index()
result_val_2.columns=['user_id', 'actual_val_2']
result_val_1['actual_val_2'] = result_val_2['actual_val_2']

In [9]:
result_val_1.head(3)

,user_id,actual_val_1,actual_val_2
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[821867, 834484, 856942, 865456, 889248, 90795..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[835476, 851057, 872021, 878302, 879948, 90963..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[920308, 926804, 946489, 1006718, 1017061, 107..."


### Фильтрующая модель 1

In [10]:
def get_multimodels_recommendations(df, rec_name_model, N=5):
    rec_name = rec_name_model[0]
    rec_model = rec_name_model[1]
    df[rec_name] = df['user_id'].apply(lambda x: rec_model(x, N=N))

In [11]:
def miltimodel_precision_at_k(df, actual_column_name, starting_column, top_n):
    for col_name in df.columns[starting_column:]:
        yield col_name, df.apply(lambda row: precision_at_k(row[col_name], row[actual_column_name], k=top_n), axis=1).mean()

In [12]:
# Модель без взвешивания
recommender_model = MainRecommender(data_train_1, weighting='')

C:\Users\vladuz\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6001 [00:00<?, ?it/s]

In [13]:
# Модель c взвешиванием bm25
recommender_model_bm25 = MainRecommender(data_train_1, weighting='bm25')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6001 [00:00<?, ?it/s]

In [14]:
# Модель c взвешиванием tfidf
recommender_model_tfidf = MainRecommender(data_train_1, weighting='tfidf')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/6001 [00:00<?, ?it/s]

In [15]:
own_rec = ('own_recs', recommender_model.get_own_recommendations)
own_rec_bm25 = ('own_recs_bm25', recommender_model_bm25.get_own_recommendations)
own_rec_tfidf = ('own_recs_tfidf', recommender_model_tfidf.get_own_recommendations)
als_rec = ('als_recs', recommender_model.get_als_recommendations)
als_rec_bm25 = ('als_recs_bm25', recommender_model_bm25.get_als_recommendations)
als_rec_tfidf = ('als_recs_tfidf', recommender_model_tfidf.get_als_recommendations)
sim_user_rec = ('similar_user_recs', recommender_model.get_similar_users_recommendation)
sim_user_rec_bm25 = ('similar_user_recs_bm25', recommender_model_bm25.get_similar_users_recommendation)
sim_user_rec_bm25 = ('similar_user_recs_tfidf', recommender_model_tfidf.get_similar_users_recommendation)
sim_item_rec = ('similar_item_recs', recommender_model.get_similar_items_recommendation)
sim_item_rec_bm25 = ('similar_item_recs_bm25', recommender_model_bm25.get_similar_items_recommendation)
sim_item_rec_tfidf = ('similar_item_recs_tfidf', recommender_model_tfidf.get_similar_items_recommendation)

In [16]:
%%time
for rec in (own_rec, own_rec_bm25, own_rec_tfidf, als_rec, als_rec_bm25, als_rec_tfidf, sim_user_rec, sim_user_rec_bm25, sim_user_rec_bm25, sim_item_rec, sim_item_rec_bm25, sim_item_rec_tfidf):
    get_multimodels_recommendations(result_val_1, rec, N=5)

CPU times: total: 2min 31s
Wall time: 1min 7s


In [17]:
result_val_1.head(3)

,user_id,actual_val_1,actual_val_2,own_recs,own_recs_bm25,own_recs_tfidf,als_recs,als_recs_bm25,als_recs_tfidf,similar_user_recs,similar_user_recs_tfidf,similar_item_recs,similar_item_recs_bm25,similar_item_recs_tfidf
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[821867, 834484, 856942, 865456, 889248, 90795...","[8090521, 1040807, 940947, 856942, 1106523]","[9655212, 877391, 1088462, 5577022, 9297615]","[856942, 8090521, 986912, 1004906, 940947]","[5569374, 986912, 1062002, 940947, 856942]","[5577022, 1088462, 856942, 883616, 9527558]","[1100972, 965766, 832678, 908318, 1102067]","[1106523, 8090537, 881391, 5569374, 5569230]","[1126899, 1132771, 5578856, 832678, 1106523]","[12298966, 5582712, 894406, 1036852, 1132231]","[912704, 1007512, 7025250, 8119134, 1132231]","[9859111, 5582712, 996259, 8119134, 920200]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[835476, 851057, 872021, 878302, 879948, 90963...","[916122, 1075368, 8090521, 5569230, 1106523]","[1118878, 9365106, 1056620, 1075832, 1076580]","[1075368, 8090521, 916122, 5569230, 1106523]","[5569230, 1106523, 8090521, 916122, 8090537]","[1014458, 1056620, 1018769, 913682, 849870]","[5569230, 1004906, 1106523, 8090521, 8090537]","[7442008, 940947, 1123106, 833241, 1106523]","[8090539, 852015, 1051283, 1094190, 1106523]","[8090537, 999999, 5569845, 819978, 999999]","[8090537, 1044078, 5569845, 819978, 985999]","[8090537, 999999, 5569845, 819978, 999999]"
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[920308, 926804, 946489, 1006718, 1017061, 107...","[6395907, 1126899, 1075368, 5569230, 1106523]","[1121367, 1137010, 936470, 6391541, 1052294]","[944534, 1075368, 902172, 5569230, 1044078]","[1075368, 844179, 5569230, 1044078, 883932]","[891423, 6391541, 936470, 1099446, 1137010]","[902172, 1075368, 965766, 844179, 907631]","[1075368, 1070845, 8090521, 899736, 1106523]","[1070272, 826666, 1031316, 850102, 1106523]","[6703856, 9297571, 960075, 1111786, 10282046]","[991932, 1082627, 960075, 1111786, 999714]","[13159268, 973135, 960075, 10356272, 963727]"


In [18]:
# N = 5
dict(sorted(miltimodel_precision_at_k(result_val_1, 'actual_val_1', 3,  top_n=5), key=lambda x: x[1], reverse=True))

{'own_recs_tfidf': 0.19100702576112413,
 'own_recs': 0.19035128805620613,
 'als_recs': 0.16608899297423887,
 'own_recs_bm25': 0.1559718969555035,
 'als_recs_bm25': 0.1476346604215457,
 'als_recs_tfidf': 0.12627634660421547,
 'similar_user_recs': 0.06997658079625292,
 'similar_user_recs_tfidf': 0.06295081967213115,
 'similar_item_recs_bm25': 0.05358313817330211,
 'similar_item_recs_tfidf': 0.046370023419203744,
 'similar_item_recs': 0.041779859484777526}

Наилучшую точность показываеют как и ожидалось модели ItemItemRecommender, а так же ALS без взвешивания после подбора параметров алгоритма ALS (в src\recommendations.py).  
Так же полсе перебора количества items при получении рекомендации - получил что N=5 наимболее оптимальный вариант, так как увеличение N дает ухудшение метрики.

### Фильтрующая модель 2

#### Подготовка датасета для обучающей выборки

Возьмем кандилатов из двух моделей - own_rec_tfidf и als_recs

In [19]:
# Возьмем пользователей из второй обучающей выборки
df_users_own_recs_tfidf = pd.DataFrame(data_train_2['user_id'].unique())
df_users_own_recs_tfidf.columns = ['user_id'] 
df_users_als_recs = df_users_own_recs_tfidf.copy()

In [20]:
# Возьмем соответствующих кондидатов из соответствующих рекоммендаций моделей
df_users_own_recs_tfidf['candidates'] = df_users_own_recs_tfidf['user_id'].apply(lambda x: recommender_model_tfidf.get_own_recommendations(x))
df_users_als_recs['candidates'] = df_users_als_recs['user_id'].apply(lambda x: recommender_model.get_als_recommendations(x, N=5))

In [21]:
df_users_own_recs_tfidf.head(3)

,user_id,candidates
0,2070,"[1126899, 1044078, 5569230, 916122, 1106523]"
1,2021,"[1004906, 5569230, 1106523, 1044078, 844179]"
2,1753,"[8090521, 1044078, 916122, 5569230, 1106523]"


In [22]:
df_users_als_recs.head(3)

,user_id,candidates
0,2070,"[866211, 1106523, 899624, 878996, 916122]"
1,2021,"[844179, 1044078, 916122, 1106523, 5569230]"
2,1753,"[1106523, 916122, 844179, 1044078, 1004906]"


Разворачиваем кандидатов, чтобы в каждой строке был только один:

In [23]:
df_items = df_users_own_recs_tfidf.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'
# Удаляем признак candidates и добавляем в датасет развернутые итемы
df_users_own_recs_tfidf = df_users_own_recs_tfidf.drop('candidates', axis=1).join(df_items)

In [24]:
df_users_own_recs_tfidf.head(3)

,user_id,item_id
0,2070,1126899
0,2070,1044078
0,2070,5569230


In [25]:
df_items = df_users_als_recs.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'
# Удаляем признак candidates и добавляем в датасет развернутые итемы
df_users_als_recs = df_users_als_recs.drop('candidates', axis=1).join(df_items)

In [26]:
df_users_als_recs.head(3)

,user_id,item_id
0,2070,866211
0,2070,1106523
0,2070,899624


Создадим обучающие датасеты для двух моделей для ранжирования с учетом кандидатов полученных от фильтрующей модели 1

In [27]:
# Создаем целевой датасет из обучающей выборки для 2 модели
targets_model_2 = data_train_2[['user_id', 'item_id', 'quantity', 'sales_value', 'store_id', 'week_no']].copy()
# Добавляем признак target отражающий наличие/отсутствие покупок
targets_model_2['target'] = 1

In [28]:
targets_model_2.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target
2104867,2070,1019940,1,1.00,311,86,1
2107468,2021,840361,1,0.99,443,86,1
2107469,2021,856060,1,1.77,443,86,1


Добавляем кандидатов модели own_recs_tfidf в датасет

In [29]:
targets_model_2_own_recs_tfidf = df_users_own_recs_tfidf.merge(targets_model_2, on=['user_id', 'item_id'], how='left')
targets_model_2_own_recs_tfidf['target'].fillna(0, inplace= True)

In [30]:
targets_model_2_own_recs_tfidf.head(4)

,user_id,item_id,quantity,sales_value,store_id,week_no,target
0,2070,1126899,NaN,NaN,NaN,NaN,0.0
1,2070,1044078,NaN,NaN,NaN,NaN,0.0
2,2070,5569230,NaN,NaN,NaN,NaN,0.0
3,2070,916122,2.0,8.76,311.0,89.0,1.0


Добавляем кандидатов модели als_recs в датасет

In [31]:
targets_model_2_als_recs = df_users_als_recs.merge(targets_model_2, on=['user_id', 'item_id'], how='left')
targets_model_2_als_recs['target'].fillna(0, inplace= True)

In [32]:
targets_model_2_als_recs.head(4)

,user_id,item_id,quantity,sales_value,store_id,week_no,target
0,2070,866211,NaN,NaN,NaN,NaN,0.0
1,2070,1106523,NaN,NaN,NaN,NaN,0.0
2,2070,899624,1.0,2.69,311.0,91.0,1.0
3,2070,878996,NaN,NaN,NaN,NaN,0.0


In [33]:
targets_model_2_own_recs_tfidf['target'].value_counts()

0.0    8636
1.0    3547
Name: target, dtype: int64

In [34]:
targets_model_2_own_recs_tfidf['target'].mean()

0.29114339653615695

In [35]:
targets_model_2_als_recs['target'].value_counts()

0.0    8902
1.0    2785
Name: target, dtype: int64

In [36]:
targets_model_2_als_recs['target'].mean()

0.2382989646615898

Дисбаланс классов присутствует но не существенный

#### Сформируем фичи для обучения модели

In [37]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [38]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_id               92353 non-null  int64 
 1   manufacturer          92353 non-null  int64 
 2   department            92353 non-null  object
 3   brand                 92353 non-null  object
 4   commodity_desc        92353 non-null  object
 5   sub_commodity_desc    92353 non-null  object
 6   curr_size_of_product  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


In [39]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [40]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age_desc             801 non-null    object
 1   marital_status_code  801 non-null    object
 2   income_desc          801 non-null    object
 3   homeowner_desc       801 non-null    object
 4   hh_comp_desc         801 non-null    object
 5   household_size_desc  801 non-null    object
 6   kid_category_desc    801 non-null    object
 7   user_id              801 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.2+ KB


In [41]:
targets_model_2_own_recs_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12183 entries, 0 to 12182
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      12183 non-null  int64  
 1   item_id      12183 non-null  int64  
 2   quantity     3547 non-null   float64
 3   sales_value  3547 non-null   float64
 4   store_id     3547 non-null   float64
 5   week_no      3547 non-null   float64
 6   target       12183 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 761.4 KB


In [42]:
targets_model_2_als_recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11687 entries, 0 to 11686
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      11687 non-null  int64  
 1   item_id      11687 non-null  int64  
 2   quantity     2785 non-null   float64
 3   sales_value  2785 non-null   float64
 4   store_id     2785 non-null   float64
 5   week_no      2785 non-null   float64
 6   target       11687 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 730.4 KB


Добавим фичи в целевой обучающие датасеты

In [43]:
# Для кандидатов модели ItemItem Recommender с взвешиванием tfidf
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(item_features, on='item_id', how='left')
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(user_features, on='user_id', how='left')

targets_model_2_own_recs_tfidf.head(2)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1126899,NaN,NaN,NaN,NaN,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1044078,NaN,NaN,NaN,NaN,0.0,2845,MEAT,National,BEEF,LEAN,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [44]:
# Для кандидатов модели ALS без взвешивания
targets_model_2_als_recs = targets_model_2_als_recs.merge(item_features, on='item_id', how='left')
targets_model_2_als_recs = targets_model_2_als_recs.merge(user_features, on='user_id', how='left')

targets_model_2_als_recs.head(2)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,866211,NaN,NaN,NaN,NaN,0.0,2,PRODUCE,National,GRAPES,GRAPES WHITE,18 LB,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1106523,NaN,NaN,NaN,NaN,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


#### Генерация признаков(фичей)

Заполним пропуски в датасетах для признаков quantity и sales_value медианами.

In [45]:
targets_model_2_own_recs_tfidf['quantity'].fillna(targets_model_2_own_recs_tfidf['quantity'].median(), inplace=True)
targets_model_2_own_recs_tfidf['sales_value'].fillna(targets_model_2_own_recs_tfidf['sales_value'].mean(), inplace=True)
targets_model_2_als_recs['quantity'].fillna(targets_model_2_als_recs['quantity'].median(), inplace=True)
targets_model_2_als_recs['sales_value'].fillna(targets_model_2_als_recs['sales_value'].mean(), inplace=True)

In [46]:
targets_model_2_own_recs_tfidf.head(2)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1044078,1.0,3.975653,NaN,NaN,0.0,2845,MEAT,National,BEEF,LEAN,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [47]:
targets_model_2_als_recs.head(2)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,GRAPES,GRAPES WHITE,18 LB,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1106523,1.0,4.026309,NaN,NaN,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


Заменим пропуски для признака store_id модой и сформируем признак mode_store_id

In [48]:
def set_store_id_mode_by_user(value_list, my_mode):
    mode_by_list = pd.Series.mode(value_list)
    if len(mode_by_list) > 0:
        return mode_by_list[0]
    else: 
        return my_mode

In [49]:
# Получим моду по всем значения признака store_id 
store_id_mode = pd.Series(targets_model_2_own_recs_tfidf['store_id']).mode()[0]

# Заменим пропуски на моду признака store_id по конкретному пользователю или в случе если для пользователя везде пропуски заменим на моду по всему store_id 
# и сохраним в отдельный датасет
temp_df = targets_model_2_own_recs_tfidf.groupby('item_id')['store_id'].agg(lambda x: set_store_id_mode_by_user(x, store_id_mode)).reset_index()
temp_df.rename(columns={'store_id': 'store_id_user_mode'}, inplace=True)

# Добавим полученные результаты в исходный датасет в новый признак store_id_user_mode
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df, on='item_id', how='inner')

In [50]:
# Получим моду по всем значения признака store_id 
store_id_mode = pd.Series(targets_model_2_als_recs['store_id']).mode()[0]

# Заменим пропуски на моду признака store_id по конкретному пользователю или в случе если для пользователя везде пропуски заменим на моду по всему store_id 
# и сохраним в отдельный датасет
temp_df = targets_model_2_als_recs.groupby('item_id')['store_id'].agg(lambda x: set_store_id_mode_by_user(x, store_id_mode)).reset_index()
temp_df.rename(columns={'store_id': 'store_id_user_mode'}, inplace=True)

# Добавим полученные результаты в исходный датасет в новый признак store_id_user_mode
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df, on='item_id', how='inner')

In [51]:
targets_model_2_own_recs_tfidf.head(2)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,367.0
1,2430,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,FLUID MILK WHITE ONLY,1 GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,367.0


In [52]:
targets_model_2_als_recs.head(2)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,GRAPES WHITE,18 LB,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,333.0
1,1346,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,GRAPES WHITE,18 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,333.0


Сгенерируем новый признак quantatity_of_item_per_week как медиану по количеству товаров приобретеннх в неделю для каждого item_id

In [53]:
# Преобразуем исходный датасет в таблицу где строки это item_id а столбцы это номер недели.
temp_df = pd.pivot_table(targets_model_2_own_recs_tfidf,
                    index='item_id', columns='week_no',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет вычислив для каждого item_id медиану покупок за неделю и сохраним данные значения в новом признаке
temp_df = temp_df.agg('median', axis=1).reset_index()
temp_df.columns = ['item_id', 'quantatity_of_item_per_week']

# Добавим новый полученный признак в исходный датасет
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df,
                                    on='item_id',
                                    how='inner')

In [54]:
# Преобразуем исходный датасет в таблицу где строки это item_id а столбцы это номер недели.
temp_df = pd.pivot_table(targets_model_2_als_recs,
                    index='item_id', columns='week_no',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет вычислив для каждого item_id медиану покупок за неделю и сохраним данные значения в новом признаке
temp_df = temp_df.agg('median', axis=1).reset_index()
temp_df.columns = ['item_id', 'quantatity_of_item_per_week']

# Добавим новый полученный признак в исходный датасет
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df,
                                    on='item_id',
                                    how='inner')

In [55]:
targets_model_2_own_recs_tfidf.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,367.0,51.5
1,2430,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,1 GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,367.0,51.5
2,2181,1126899,1.0,2.790000,401.0,91.0,1.0,69,GROCERY,Private,...,1 GA,55-64,A,100-124K,Homeowner,2 Adults Kids,3,1,367.0,51.5


In [56]:
targets_model_2_als_recs.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,18 LB,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,333.0,13.5
1,1346,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,18 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,333.0,13.5
2,2324,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,18 LB,35-44,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,333.0,13.5


Сгенерируем новый признак quantatity_of_item_in_category_per_week как медиану по количеству товаров приобретеннх за неделю в каждой категории

In [57]:
# Преобразуем исходный датасет в таблицу где строки это категории а столбцы это номер недели.
temp_df = pd.pivot_table(targets_model_2_own_recs_tfidf,
                    index='department', columns='week_no',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет вычислив для каждой категории медиану покупок за неделю и сохраним данные значения в новом признаке
temp_df = temp_df.agg('median', axis=1).reset_index()
temp_df.columns = ['department', 'quantatity_of_item_in_category_per_week']

# Добавим новый полученный признак в исходный датасет
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df,
                                    on='department',
                                    how='inner')

In [58]:
# Преобразуем исходный датасет в таблицу где строки это категории а столбцы это номер недели.
temp_df = pd.pivot_table(targets_model_2_als_recs,
                    index='department', columns='week_no',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет вычислив для каждой категории медиану покупок за неделю и сохраним данные значения в новом признаке
temp_df = temp_df.agg('median', axis=1).reset_index()
temp_df.columns = ['department', 'quantatity_of_item_in_category_per_week']

# Добавим новый полученный признак в исходный датасет
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df,
                                    on='department',
                                    how='inner')

In [59]:
targets_model_2_own_recs_tfidf.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,367.0,51.5,357.0
1,2430,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,367.0,51.5,357.0
2,2181,1126899,1.0,2.790000,401.0,91.0,1.0,69,GROCERY,Private,...,55-64,A,100-124K,Homeowner,2 Adults Kids,3,1,367.0,51.5,357.0


In [60]:
targets_model_2_als_recs.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,333.0,13.5,80.5
1,1346,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,333.0,13.5,80.5
2,2324,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,35-44,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,333.0,13.5,80.5


Сгенерируем новый признак top_categories как категорию в которой было совершео больше всего покупок пользователем

In [61]:
# Преобразуем исходный датасет в таблицу где строки это пользователи а столбцы это категории товаров.
temp_df = pd.pivot_table(targets_model_2_own_recs_tfidf,
                    index='user_id', columns='department',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет определив для каждого полльзователя категорию в которй им было сделано больше всего покупок
temp_df = temp_df.idxmax(axis=1).reset_index()
temp_df.columns = ['user_id', 'top_categories']

# Добавим новый полученный признак в исходный датасет
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df,
                                    on='user_id',
                                    how='inner')

In [62]:
# Преобразуем исходный датасет в таблицу где строки это пользователи а столбцы это категории товаров.
temp_df = pd.pivot_table(targets_model_2_als_recs,
                    index='user_id', columns='department',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет определив для каждого полльзователя категорию в которй им было сделано больше всего покупок
temp_df = temp_df.idxmax(axis=1).reset_index()
temp_df.columns = ['user_id', 'top_categories']

# Добавим новый полученный признак в исходный датасет
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df,
                                    on='user_id',
                                    how='inner')

In [63]:
targets_model_2_own_recs_tfidf.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,U,50-74K,Unknown,Unknown,1,None/Unknown,367.0,51.5,357.0,GROCERY
1,2070,5569230,1.0,3.975653,NaN,NaN,0.0,1208,GROCERY,National,...,U,50-74K,Unknown,Unknown,1,None/Unknown,439.0,32.5,357.0,GROCERY
2,2070,1106523,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,U,50-74K,Unknown,Unknown,1,None/Unknown,292.0,127.5,357.0,GROCERY


In [64]:
targets_model_2_als_recs.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,U,50-74K,Unknown,Unknown,1,None/Unknown,333.0,13.5,80.5,PRODUCE
1,2070,899624,1.0,2.690000,311.0,91.0,1.0,69,PRODUCE,Private,...,U,50-74K,Unknown,Unknown,1,None/Unknown,443.0,11.5,80.5,PRODUCE
2,2070,878996,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,U,50-74K,Unknown,Unknown,1,None/Unknown,304.0,3.5,80.5,PRODUCE


Сгенерируем новый признак top_brands котражающий максимально популярный для каждого пользователя бренд производетеля товаров

In [65]:
# Преобразуем исходный датасет в таблицу где строки это пользователи а столбцы это категория производителя товаров.
temp_df = pd.pivot_table(targets_model_2_own_recs_tfidf,
                    index='user_id', columns='brand',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет определив для каждого полльзователя наиболее популярную категорию производителя товаров
temp_df = temp_df.idxmax(axis=1).reset_index()
temp_df.columns = ['user_id', 'top_brands']

# Добавим новый полученный признак в исходный датасет
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df,
                                    on='user_id',
                                    how='inner')

In [66]:
# Преобразуем исходный датасет в таблицу где строки это пользователи а столбцы это категория производителя товаров.
temp_df = pd.pivot_table(targets_model_2_als_recs,
                    index='user_id', columns='brand',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

# Преобразуем полученный датасет определив для каждого полльзователя наиболее популярную категорию производителя товаров
temp_df = temp_df.idxmax(axis=1).reset_index()
temp_df.columns = ['user_id', 'top_brands']

# Добавим новый полученный признак в исходный датасет
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df,
                                    on='user_id',
                                    how='inner')

In [67]:
targets_model_2_own_recs_tfidf.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories,top_brands
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,50-74K,Unknown,Unknown,1,None/Unknown,367.0,51.5,357.0,GROCERY,National
1,2070,5569230,1.0,3.975653,NaN,NaN,0.0,1208,GROCERY,National,...,50-74K,Unknown,Unknown,1,None/Unknown,439.0,32.5,357.0,GROCERY,National
2,2070,1106523,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,50-74K,Unknown,Unknown,1,None/Unknown,292.0,127.5,357.0,GROCERY,National


In [68]:
targets_model_2_als_recs.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories,top_brands
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,50-74K,Unknown,Unknown,1,None/Unknown,333.0,13.5,80.5,PRODUCE,National
1,2070,899624,1.0,2.690000,311.0,91.0,1.0,69,PRODUCE,Private,...,50-74K,Unknown,Unknown,1,None/Unknown,443.0,11.5,80.5,PRODUCE,National
2,2070,878996,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,50-74K,Unknown,Unknown,1,None/Unknown,304.0,3.5,80.5,PRODUCE,National


Сгенерируем новый признак mean_sales_value_of_user_in_category отражающий средний обьем закупок пользователя по категории товаров

In [69]:
# Преобразуем исходный датасет в таблицу где строки это пользователи а столбцы это категории.
# Значение в ячейках это средний обьем закупок пользователя в каждой категории
temp_df = pd.pivot_table(targets_model_2_own_recs_tfidf,
                    index='user_id', columns='department',
                    values='sales_value',
                    aggfunc='mean',
                    fill_value=0
                    )

# Преобразуем полученный датасет определив для каждого полльзователя наиболее популярную категорию производителя товаров
temp_df = temp_df.stack().reset_index()
temp_df.columns = ['user_id', 'department', 'mean_sales_value_of_user_in_category']

# Добавим новый полученный признак в исходный датасет
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df,
                                    on=['user_id', 'department'],
                                    how='inner')

In [70]:
# Преобразуем исходный датасет в таблицу где строки это пользователи а столбцы это категории.
# Значение в ячейках это средний обьем закупок пользователя в каждой категории
temp_df = pd.pivot_table(targets_model_2_als_recs,
                    index='user_id', columns='department',
                    values='sales_value',
                    aggfunc='mean',
                    fill_value=0
                    )

# Преобразуем полученный датасет определив для каждого полльзователя средний обьем закупок товаров по категориям или средний обьем продаж по категории для каждого пользователя
temp_df = temp_df.stack().reset_index()
temp_df.columns = ['user_id', 'department', 'mean_sales_value_of_user_in_category']

# Добавим новый полученный признак в исходный датасет
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df,
                                    on=['user_id', 'department'],
                                    how='inner')

In [71]:
targets_model_2_own_recs_tfidf.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories,top_brands,mean_sales_value_of_user_in_category
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,Unknown,Unknown,1,None/Unknown,367.0,51.5,357.0,GROCERY,National,3.975653
1,2070,5569230,1.0,3.975653,NaN,NaN,0.0,1208,GROCERY,National,...,Unknown,Unknown,1,None/Unknown,439.0,32.5,357.0,GROCERY,National,3.975653
2,2070,1106523,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,Unknown,Unknown,1,None/Unknown,292.0,127.5,357.0,GROCERY,National,3.975653


In [72]:
targets_model_2_als_recs.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories,top_brands,mean_sales_value_of_user_in_category
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,Unknown,Unknown,1,None/Unknown,333.0,13.5,80.5,PRODUCE,National,3.580873
1,2070,899624,1.0,2.690000,311.0,91.0,1.0,69,PRODUCE,Private,...,Unknown,Unknown,1,None/Unknown,443.0,11.5,80.5,PRODUCE,National,3.580873
2,2070,878996,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,Unknown,Unknown,1,None/Unknown,304.0,3.5,80.5,PRODUCE,National,3.580873


Заменим пропуски в признаке age_desc на моду по каждму пользователю и создадим новый признак age_desc_full.

In [73]:
temp_df = targets_model_2_own_recs_tfidf.groupby(by=['user_id'])['age_desc'].apply(lambda x: pd.Series.mode(x))
temp_df = temp_df.reset_index()
temp_df.drop(columns='level_1', inplace=True)
temp_df.columns=['user_id', 'age_desc_full']
targets_model_2_own_recs_tfidf = targets_model_2_own_recs_tfidf.merge(temp_df, on='user_id', how='inner')

In [74]:
temp_df = targets_model_2_als_recs.groupby(by=['user_id'])['age_desc'].apply(lambda x: pd.Series.mode(x))
temp_df = temp_df.reset_index()
temp_df.drop(columns='level_1', inplace=True)
temp_df.columns=['user_id', 'age_desc_full']
targets_model_2_als_recs = targets_model_2_als_recs.merge(temp_df, on='user_id', how='inner')

In [75]:
targets_model_2_own_recs_tfidf.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories,top_brands,mean_sales_value_of_user_in_category,age_desc_full
0,2070,1126899,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,Unknown,1,None/Unknown,367.0,51.5,357.0,GROCERY,National,3.975653,45-54
1,2070,5569230,1.0,3.975653,NaN,NaN,0.0,1208,GROCERY,National,...,Unknown,1,None/Unknown,439.0,32.5,357.0,GROCERY,National,3.975653,45-54
2,2070,1106523,1.0,3.975653,NaN,NaN,0.0,69,GROCERY,Private,...,Unknown,1,None/Unknown,292.0,127.5,357.0,GROCERY,National,3.975653,45-54


In [76]:
targets_model_2_als_recs.head(3)

,user_id,item_id,quantity,sales_value,store_id,week_no,target,manufacturer,department,brand,...,hh_comp_desc,household_size_desc,kid_category_desc,store_id_user_mode,quantatity_of_item_per_week,quantatity_of_item_in_category_per_week,top_categories,top_brands,mean_sales_value_of_user_in_category,age_desc_full
0,2070,866211,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,Unknown,1,None/Unknown,333.0,13.5,80.5,PRODUCE,National,3.580873,45-54
1,2070,899624,1.0,2.690000,311.0,91.0,1.0,69,PRODUCE,Private,...,Unknown,1,None/Unknown,443.0,11.5,80.5,PRODUCE,National,3.580873,45-54
2,2070,878996,1.0,4.026309,NaN,NaN,0.0,2,PRODUCE,National,...,Unknown,1,None/Unknown,304.0,3.5,80.5,PRODUCE,National,3.580873,45-54


### Сформируем финальные датасеты для обучения

In [77]:
targets_model_2_own_recs_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4812 entries, 0 to 4811
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   user_id                                  4812 non-null   int64  
 1   item_id                                  4812 non-null   int64  
 2   quantity                                 4812 non-null   float64
 3   sales_value                              4812 non-null   float64
 4   store_id                                 1969 non-null   float64
 5   week_no                                  1969 non-null   float64
 6   target                                   4812 non-null   float64
 7   manufacturer                             4812 non-null   int64  
 8   department                               4812 non-null   object 
 9   brand                                    4812 non-null   object 
 10  commodity_desc                           4812 no

In [78]:
targets_model_2_own_recs_tfidf.drop(['store_id', 'week_no'], axis=1, inplace=True)

In [79]:
targets_model_2_own_recs_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4812 entries, 0 to 4811
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   user_id                                  4812 non-null   int64  
 1   item_id                                  4812 non-null   int64  
 2   quantity                                 4812 non-null   float64
 3   sales_value                              4812 non-null   float64
 4   target                                   4812 non-null   float64
 5   manufacturer                             4812 non-null   int64  
 6   department                               4812 non-null   object 
 7   brand                                    4812 non-null   object 
 8   commodity_desc                           4812 non-null   object 
 9   sub_commodity_desc                       4812 non-null   object 
 10  curr_size_of_product                     4812 no

In [80]:
targets_model_2_own_recs_tfidf.columns

Index(['user_id', 'item_id', 'quantity', 'sales_value', 'target',
       'manufacturer', 'department', 'brand', 'commodity_desc',
       'sub_commodity_desc', 'curr_size_of_product', 'age_desc',
       'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc',
       'household_size_desc', 'kid_category_desc', 'store_id_user_mode',
       'quantatity_of_item_per_week',
       'quantatity_of_item_in_category_per_week', 'top_categories',
       'top_brands', 'mean_sales_value_of_user_in_category', 'age_desc_full'],
      dtype='object')

In [81]:
categorical_features = ['department', 'brand', 'commodity_desc',
                        'sub_commodity_desc', 'curr_size_of_product', 'age_desc',
                        'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc',
                        'household_size_desc', 'kid_category_desc', 'top_categories',
                        'top_brands', 'age_desc_full']

In [82]:
targets_model_2_own_recs_tfidf[categorical_features] = targets_model_2_own_recs_tfidf[categorical_features].astype('category')
targets_model_2_als_recs[categorical_features] = targets_model_2_als_recs[categorical_features].astype('category')

In [83]:
train_2_features = ['user_id', 'item_id', 'quantity', 
                    'sales_value', 'department', 'manufacturer', 
                    'age_desc_full', 'brand',
                    'store_id_user_mode', 'quantatity_of_item_per_week',
                    'quantatity_of_item_in_category_per_week', 
                    'top_categories', 'top_brands', 
                    'mean_sales_value_of_user_in_category']

train_2_cat_features = ['department', 'brand', 'age_desc_full',
                        'top_categories', 'top_brands']

In [84]:
X_train_2_own_recs_tfidf = targets_model_2_own_recs_tfidf[train_2_features]
y_train_2_own_recs_tfidf = targets_model_2_own_recs_tfidf['target']
X_train_2_als_recs = targets_model_2_als_recs[train_2_features]
y_train_2_als_recs = targets_model_2_als_recs['target']

### Обучение модели 2

In [85]:
model_own_tfidf = CatBoostClassifier(random_seed=45, iterations=100, learning_rate=0.1)
model_als = CatBoostClassifier(random_seed=45, iterations=100, learning_rate=0.1)

In [86]:
model_own_tfidf.fit(X_train_2_own_recs_tfidf, y_train_2_own_recs_tfidf, cat_features=train_2_cat_features, verbose=50)

0:	learn: 0.6026305	total: 85.8ms	remaining: 8.49s
50:	learn: 0.0074449	total: 2.21s	remaining: 2.13s
99:	learn: 0.0013698	total: 4.24s	remaining: 0us


In [87]:
model_als.fit(X_train_2_als_recs, y_train_2_als_recs, cat_features=train_2_cat_features, verbose=50)

0:	learn: 0.6025738	total: 31.2ms	remaining: 3.09s
50:	learn: 0.0075834	total: 2.09s	remaining: 2.01s
99:	learn: 0.0013643	total: 3.95s	remaining: 0us


In [88]:
train_preds_own_tfidf = model_own_tfidf.predict(X_train_2_own_recs_tfidf)
train_preds_own_tfidf = train_preds_own_tfidf.astype(bool)

train_preds_als = model_als.predict(X_train_2_als_recs)
train_preds_als = train_preds_als.astype(bool)

In [89]:
items_2_own_recs_tfidf = X_train_2_own_recs_tfidf[train_preds_own_tfidf].groupby(by=['user_id'])['item_id'].unique().reset_index()
items_2_own_recs_tfidf.columns = ['user_id', 'model_own_recs_tfidf_preds']

items_2_als_recs = X_train_2_als_recs[train_preds_als].groupby(by=['user_id'])['item_id'].unique().reset_index()
items_2_als_recs.columns = ['user_id', 'model_als_recs_preds']

In [90]:
result_lvl_2 = data_val_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns = ['user_id', 'actual']
result_lvl_2.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [91]:
result_lvl_2 = result_lvl_2.merge(items_2_own_recs_tfidf,
                                  on='user_id',
                                  how='inner')

result_lvl_2 = result_lvl_2.merge(items_2_als_recs,
                                  on='user_id',
                                  how='inner')

In [92]:
result_lvl_2

,user_id,actual,model_own_recs_tfidf_preds,model_als_recs_preds
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 1004906, 940947]","[856942, 940947]"
1,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1126899, 1106523, 1122358]","[1106523, 1126899, 1122358]"
2,17,"[892503, 896085, 913806, 923559, 928036, 93748...","[1106523, 1044078, 916122]","[899624, 916122, 1044078]"
3,18,"[831628, 907877, 914697, 995242, 1118878, 1128...",[916122],"[951412, 916122]"
4,20,"[819112, 944419, 945611, 971684, 1025522, 1058...","[1126899, 8090536, 1075368]","[1126899, 8090536, 944836, 9835509, 1075368]"
...,...,...,...,...
429,2486,"[821219, 870195, 959172, 976998, 995242, 55692...","[8090521, 916122]",[916122]
430,2488,"[6534178, 820165, 826249, 836163, 891961, 9068...","[1070820, 844179]","[834117, 913210, 865456]"
431,2496,[6534178],"[5569230, 1106523, 916122]","[916122, 12810393]"
432,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1106523, 1070820]",[1070820]


In [93]:
result_lvl_2.apply(lambda row: precision_at_k(row['model_own_recs_tfidf_preds'], row['actual']), axis=1).mean()

0.49807987711213514

In [94]:
result_lvl_2.apply(lambda row: precision_at_k(row['model_als_recs_preds'], row['actual']), axis=1).mean()

0.44723502304147467

In [95]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual_test']
result_test.head(2)

,user_id,actual_test
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [96]:
result_test = result_test.merge(items_2_own_recs_tfidf,
                                  on='user_id',
                                  how='inner')

result_test = result_test.merge(items_2_als_recs,
                                  on='user_id',
                                  how='inner')

In [102]:
result_test.head(5)

,user_id,model_preds
0,1,"[856942, 1004906, 940947]"
1,7,"[1126899, 1106523, 1122358]"
2,18,[916122]
3,20,"[1126899, 8090536, 1075368]"
4,22,[1070820]


In [98]:
result_test.apply(lambda row: precision_at_k(row['model_own_recs_tfidf_preds'], row['actual_test']), axis=1).mean()

0.36709803921568623

In [99]:
result_test.apply(lambda row: precision_at_k(row['model_als_recs_preds'], row['actual_test']), axis=1).mean()

0.33741176470588236

Выгружаем лучшую модель в файл

In [100]:
result_test.drop(['actual_test', 'model_als_recs_preds'], axis=1, inplace=True)
result_test.rename(columns={"model_own_recs_tfidf_preds": "model_preds"}, inplace=True)

In [101]:
result_test.to_csv('recommendations.csv', sep='|', index=False)